In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
sub='/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv'
submission_file = pd.read_csv(sub)
submission_file.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [41]:
tra='/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
train_data=pd.read_csv(tra)
train_data


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [43]:
train_data=train_data.drop(['Id','Alley','PoolQC','MiscFeature'], axis=1)

train_data.info()

In [46]:
y_ = train_data.iloc[:,-1].values
x_ = train_data.iloc[:,:-1].values
y = pd.DataFrame(y_)
x = pd.DataFrame(x_)
x

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,MnPrv,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,GdPrv,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,NaN,0,4,2010,WD,Normal


In [47]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1460 non-null   object
 1   1       1460 non-null   object
 2   2       1201 non-null   object
 3   3       1460 non-null   object
 4   4       1460 non-null   object
 5   5       1460 non-null   object
 6   6       1460 non-null   object
 7   7       1460 non-null   object
 8   8       1460 non-null   object
 9   9       1460 non-null   object
 10  10      1460 non-null   object
 11  11      1460 non-null   object
 12  12      1460 non-null   object
 13  13      1460 non-null   object
 14  14      1460 non-null   object
 15  15      1460 non-null   object
 16  16      1460 non-null   object
 17  17      1460 non-null   object
 18  18      1460 non-null   object
 19  19      1460 non-null   object
 20  20      1460 non-null   object
 21  21      1460 non-null   object
 22  22      1460 non-null   

In [ ]:
x=pd.get_dummies(data=x, columns=[1,4,5,6,7,8,9,10,11,12,13,14,19,20,21,22,23,25,26,27,28,29,30,31,33,37,38,39,40,51,53,55,57,58,60,63,64,65,72,73,74,78,79],dummy_na=False)
x

In [24]:
x.fillna(0)

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,MnPrv,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,GdPrv,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,0,4,2010,WD,Normal


In [34]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1460 non-null   object
 1   1       1460 non-null   object
 2   2       1201 non-null   object
 3   3       1460 non-null   object
 4   4       1460 non-null   object
 5   5       1460 non-null   object
 6   6       1460 non-null   object
 7   7       1460 non-null   object
 8   8       1460 non-null   object
 9   9       1460 non-null   object
 10  10      1460 non-null   object
 11  11      1460 non-null   object
 12  12      1460 non-null   object
 13  13      1460 non-null   object
 14  14      1460 non-null   object
 15  15      1460 non-null   object
 16  16      1460 non-null   object
 17  17      1460 non-null   object
 18  18      1460 non-null   object
 19  19      1460 non-null   object
 20  20      1460 non-null   object
 21  21      1460 non-null   object
 22  22      1460 non-null   

In [22]:
#Label Encoding for object to numeric conversion

x=pd.get_dummies(data=x)
x

,0_20,0_30,0_40,0_45,0_50,0_60,0_70,0_75,0_80,0_85,...,74_ConLw,74_New,74_Oth,74_WD,75_Abnorml,75_AdjLand,75_Alloca,75_Family,75_Normal,75_Partial
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1456,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1458,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [57]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x,y)


LinearRegression()

In [103]:
test='/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
test_data=pd.read_csv(test)
test_data



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [112]:
p=test_data.Id.values
p

AttributeError: 'numpy.ndarray' object has no attribute 'Id'

In [105]:
test_data=test_data.drop(['Id','Alley','PoolQC','MiscFeature'], axis=1)

In [106]:
test_data.fillna(0)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,120,0,MnPrv,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,MnPrv,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,144,0,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,MnPrv,700,7,2006,WD,Normal


In [107]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
ob = test_data.select_dtypes(include = "object").columns

for feat in ob:
    test_data[feat] = lb.fit_transform(test_data[feat].astype(str))
test_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,2,80.0,11622,1,3,3,0,4,0,...,0,0,120,0,2,0,6,2010,8,4
1,20,3,81.0,14267,1,0,3,0,0,0,...,0,0,0,0,4,12500,6,2010,8,4
2,60,3,74.0,13830,1,0,3,0,4,0,...,0,0,0,0,2,0,3,2010,8,4
3,60,3,78.0,9978,1,0,3,0,4,0,...,0,0,0,0,4,0,6,2010,8,4
4,120,3,43.0,5005,1,0,1,0,4,0,...,0,0,144,0,4,0,1,2010,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,4,21.0,1936,1,3,3,0,4,0,...,0,0,0,0,4,0,6,2006,8,4
1455,160,4,21.0,1894,1,3,3,0,4,0,...,0,0,0,0,4,0,4,2006,8,0
1456,20,3,160.0,20000,1,3,3,0,4,0,...,0,0,0,0,4,0,9,2006,8,0
1457,85,3,62.0,10441,1,3,3,0,4,0,...,0,0,0,0,2,700,7,2006,8,4


In [108]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
test_data = my_imputer.fit_transform(test_data)

In [109]:
q=reg.predict(test_data)


In [1]:
q[0]

NameError: name 'q' is not defined

In [124]:
f=[]
for i in q:
    f.append(q[i])

In [126]:
f

[0      -2.055375e+06
 1      -7.574014e+06
 2      -1.933845e+06
 3      -1.900211e+06
 4      -1.826462e+06
             ...     
 1454   -1.881574e+06
 1455   -1.890780e+06
 1456   -2.039462e+06
 1457   -2.291163e+06
 1458   -1.860157e+06
 Name: 0, Length: 1459, dtype: float64]

In [125]:
#dic={'Id': p}
dic={'Id': p,'SalePrice':f}
re=pd.DataFrame(dic)
re

ValueError: array length 1 does not match index length 1459

In [114]:
re.to_csv('submission.csv', index=False)

NameError: name 're' is not defined